In [1]:
import random, gc, os, pickle, csv, time, re

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Ablation 1: No Sort Adapt Score
This Ablation focuses on no sorting of adapt score. So during query should be random sort.

There are a total of 4 changes:
1. Training cls_oml_ori_v2.py: Random Score on Sample Support (no_support_priority=False)
2. Training cls_oml_ori_v2.py: No memory write of update adapt score (memory.update_meta)
3. Memory base_models_ori.py: Write filter condition don’t care (sorted_support >= 1)
4. Testing: No Sample Score (sort_score=False)


# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


BASE_MODEL_PATH = "/data/model_runs/original_oml/aMEL-dssplit-order4"
find_path = "id4"

model_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*\.pt$", x)][0]
model_path = os.path.join(BASE_MODEL_PATH, model_name)
memory_name = [x for x in os.listdir(BASE_MODEL_PATH) if  re.search(f"{find_path}.*memory\.pickle$", x)][0]
memory_path = os.path.join(BASE_MODEL_PATH, memory_name)
print(f"Using model from {model_path}")
print(f"Using memory_path from {memory_path}")

use_db_cache = True
cache_dir = 'tmp'

Using model from /data/model_runs/original_oml/aMEL-dssplit-order4/OML-order4-id4-2023-09-29_03-37-26.455645.pt
Using memory_path from /data/model_runs/original_oml/aMEL-dssplit-order4/OML-order4-id4-2023-09-29_03-37-26.455679_memory.pickle


In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*5,
    "meta_lr": 3e-5,
    "model": "roberta",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 0.1,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 8000,
    "task_aware": True,
    "reverse_support": True,
    "curriculum_replay": True,
    "pln": "1fc"
}
sort_score = False # OML-ER doesn't have any score in the first place.
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-11-12 09:35:16,921 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
2023-11-12 09:35:16,924 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
2023-11-12 09:35:18,254 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c2

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [9]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=sort_score)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            if updates > 0: # You can remove this if, but then it's just ugly errors.
                acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)
                print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                            'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [10]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.6737, accuracy = 0.7250, precision = 0.7215, recall = 0.7133, F1 score = 0.7013
Query set metrics: Loss = 0.7147, accuracy = 0.7500, precision = 0.8000, recall = 0.7867, F1 score = 0.7790
Query ID 1/475
Support set metrics: Loss = 0.8677, accuracy = 0.6375, precision = 0.6440, recall = 0.6267, F1 score = 0.6227
Query set metrics: Loss = 0.8364, accuracy = 0.7500, precision = 0.7267, recall = 0.7933, F1 score = 0.7232
Query ID 2/475
Support set metrics: Loss = 0.8156, accuracy = 0.6750, precision = 0.6867, recall = 0.6600, F1 score = 0.6623
Query set metrics: Loss = 0.8264, accuracy = 0.6875, precision = 0.6833, recall = 0.7167, F1 score = 0.6910
Query ID 3/475
Support set metrics: Loss = 0.8571, accuracy = 0.6000, precision = 0.6050, recall = 0.5900, F1 score = 0.5871
Query set metrics: Loss = 1.2467, accuracy = 0.4375, precision = 0.3333, recall = 0.3833, F1 score = 

Support set metrics: Loss = 0.7664, accuracy = 0.7000, precision = 0.7003, recall = 0.6900, F1 score = 0.6869
Query set metrics: Loss = 0.6657, accuracy = 0.7500, precision = 0.7625, recall = 0.7625, F1 score = 0.7625
Query ID 36/475
Support set metrics: Loss = 0.7212, accuracy = 0.7125, precision = 0.7249, recall = 0.7067, F1 score = 0.7084
Query set metrics: Loss = 1.0171, accuracy = 0.5625, precision = 0.5667, recall = 0.4500, F1 score = 0.4981
Query ID 37/475
Support set metrics: Loss = 0.7756, accuracy = 0.6875, precision = 0.6901, recall = 0.6867, F1 score = 0.6812
Query set metrics: Loss = 0.6460, accuracy = 0.7500, precision = 0.6100, recall = 0.6267, F1 score = 0.6070
Query ID 38/475
Support set metrics: Loss = 0.7895, accuracy = 0.7375, precision = 0.7559, recall = 0.7333, F1 score = 0.7352
Query set metrics: Loss = 0.8296, accuracy = 0.6875, precision = 0.7000, recall = 0.7033, F1 score = 0.6976
Query ID 39/475
Support set metrics: Loss = 0.7540, accuracy = 0.7375, precision

Support set metrics: Loss = 0.7455, accuracy = 0.7500, precision = 0.7720, recall = 0.7367, F1 score = 0.7319
Query set metrics: Loss = 0.7932, accuracy = 0.6875, precision = 0.6600, recall = 0.6000, F1 score = 0.5792
Query ID 72/475
Support set metrics: Loss = 0.7543, accuracy = 0.6625, precision = 0.6723, recall = 0.6600, F1 score = 0.6596
Query set metrics: Loss = 0.7306, accuracy = 0.6875, precision = 0.6867, recall = 0.7700, F1 score = 0.7018
Query ID 73/475
Support set metrics: Loss = 0.7733, accuracy = 0.7000, precision = 0.6867, recall = 0.6967, F1 score = 0.6876
Query set metrics: Loss = 0.9230, accuracy = 0.6875, precision = 0.6167, recall = 0.5933, F1 score = 0.5912
Query ID 74/475
Support set metrics: Loss = 0.6809, accuracy = 0.6750, precision = 0.6576, recall = 0.6633, F1 score = 0.6537
Query set metrics: Loss = 0.8793, accuracy = 0.6875, precision = 0.7333, recall = 0.7167, F1 score = 0.7133
Query ID 75/475
Support set metrics: Loss = 0.8786, accuracy = 0.5750, precision

Support set metrics: Loss = 0.7357, accuracy = 0.6625, precision = 0.6676, recall = 0.6533, F1 score = 0.6544
Query set metrics: Loss = 1.0981, accuracy = 0.3125, precision = 0.2083, recall = 0.4333, F1 score = 0.2535
Query ID 107/475
Support set metrics: Loss = 0.7912, accuracy = 0.7500, precision = 0.7527, recall = 0.7467, F1 score = 0.7460
Query set metrics: Loss = 0.9092, accuracy = 0.5625, precision = 0.6133, recall = 0.6000, F1 score = 0.5765
Query ID 108/475
Support set metrics: Loss = 0.7047, accuracy = 0.7375, precision = 0.7373, recall = 0.7367, F1 score = 0.7344
Query set metrics: Loss = 0.8959, accuracy = 0.5625, precision = 0.6524, recall = 0.6733, F1 score = 0.5676
Query ID 109/475
Support set metrics: Loss = 0.8432, accuracy = 0.6750, precision = 0.6874, recall = 0.6767, F1 score = 0.6744
Query set metrics: Loss = 0.9508, accuracy = 0.4375, precision = 0.4367, recall = 0.4367, F1 score = 0.4343
Query ID 110/475
Support set metrics: Loss = 0.8094, accuracy = 0.6375, preci

Support set metrics: Loss = 0.8676, accuracy = 0.6500, precision = 0.6798, recall = 0.6433, F1 score = 0.6443
Query set metrics: Loss = 0.7495, accuracy = 0.6250, precision = 0.6000, recall = 0.6000, F1 score = 0.5886
Query ID 142/475
Support set metrics: Loss = 0.7314, accuracy = 0.7250, precision = 0.7341, recall = 0.7200, F1 score = 0.7241
Query set metrics: Loss = 0.7725, accuracy = 0.7500, precision = 0.6167, recall = 0.6000, F1 score = 0.6032
Query ID 143/475
Support set metrics: Loss = 0.8767, accuracy = 0.6875, precision = 0.7005, recall = 0.6733, F1 score = 0.6667
Query set metrics: Loss = 0.7785, accuracy = 0.6250, precision = 0.7667, recall = 0.7033, F1 score = 0.6233
Query ID 144/475
Support set metrics: Loss = 0.6743, accuracy = 0.7625, precision = 0.7804, recall = 0.7567, F1 score = 0.7529
Query set metrics: Loss = 1.0532, accuracy = 0.5625, precision = 0.4200, recall = 0.4833, F1 score = 0.4267
Query ID 145/475
Support set metrics: Loss = 0.7232, accuracy = 0.7375, preci

...

Support set metrics: Loss = 0.8581, accuracy = 0.7125, precision = 0.7613, recall = 0.6900, F1 score = 0.7031
Query set metrics: Loss = 0.8290, accuracy = 0.8125, precision = 0.9062, recall = 0.8750, F1 score = 0.8571
Query ID 313/475
Support set metrics: Loss = 0.9805, accuracy = 0.7500, precision = 0.7843, recall = 0.7600, F1 score = 0.7615
Query set metrics: Loss = 0.5593, accuracy = 0.8750, precision = 0.8958, recall = 0.9333, F1 score = 0.9028
Query ID 314/475
Support set metrics: Loss = 0.8838, accuracy = 0.7000, precision = 0.7487, recall = 0.6900, F1 score = 0.7005
Query set metrics: Loss = 0.6583, accuracy = 0.8750, precision = 0.7810, recall = 0.8214, F1 score = 0.7923
Query ID 315/475
Support set metrics: Loss = 1.0344, accuracy = 0.6750, precision = 0.7120, recall = 0.7000, F1 score = 0.6888
Query set metrics: Loss = 1.1072, accuracy = 0.6250, precision = 0.7143, recall = 0.6429, F1 score = 0.6429
Query ID 316/475
Support set metrics: Loss = 1.0041, accuracy = 0.6750, preci

Support set metrics: Loss = 0.6905, accuracy = 0.8125, precision = 0.8670, recall = 0.7900, F1 score = 0.8158
Query set metrics: Loss = 0.8234, accuracy = 0.6875, precision = 0.5208, recall = 0.5625, F1 score = 0.5375
Query ID 348/475
Support set metrics: Loss = 1.0372, accuracy = 0.7500, precision = 0.8263, recall = 0.7500, F1 score = 0.7749
Query set metrics: Loss = 1.0405, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 1.1577, accuracy = 0.6375, precision = 0.6885, recall = 0.6400, F1 score = 0.6445
Query set metrics: Loss = 1.2394, accuracy = 0.6250, precision = 0.9286, recall = 0.7500, F1 score = 0.7667
Query ID 350/475
Support set metrics: Loss = 0.9271, accuracy = 0.7125, precision = 0.7144, recall = 0.6800, F1 score = 0.6878
Query set metrics: Loss = 0.7002, accuracy = 0.8750, precision = 0.8500, recall = 0.8333, F1 score = 0.8361
Query ID 351/475
Support set metrics: Loss = 1.0891, accuracy = 0.6500, preci

Support set metrics: Loss = 1.0858, accuracy = 0.7375, precision = 0.8139, recall = 0.7500, F1 score = 0.7475
Query set metrics: Loss = 0.9084, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.9348, accuracy = 0.7375, precision = 0.7576, recall = 0.7400, F1 score = 0.7297
Query set metrics: Loss = 0.8288, accuracy = 0.8125, precision = 0.6905, recall = 0.7810, F1 score = 0.7175
Query ID 384/475
Support set metrics: Loss = 1.2987, accuracy = 0.6125, precision = 0.6988, recall = 0.6000, F1 score = 0.5989
Query set metrics: Loss = 1.4932, accuracy = 0.5625, precision = 0.4417, recall = 0.6167, F1 score = 0.4833
Query ID 385/475
Support set metrics: Loss = 0.9671, accuracy = 0.7125, precision = 0.7447, recall = 0.6900, F1 score = 0.7019
Query set metrics: Loss = 1.3508, accuracy = 0.6250, precision = 0.6875, recall = 0.6562, F1 score = 0.6321
Query ID 386/475
Support set metrics: Loss = 1.0878, accuracy = 0.6250, preci

Support set metrics: Loss = 1.1403, accuracy = 0.7000, precision = 0.7227, recall = 0.6900, F1 score = 0.6876
Query set metrics: Loss = 0.9381, accuracy = 0.6875, precision = 0.7250, recall = 0.7500, F1 score = 0.7000
Query ID 418/475
Support set metrics: Loss = 0.9164, accuracy = 0.7250, precision = 0.8012, recall = 0.7200, F1 score = 0.7289
Query set metrics: Loss = 1.0622, accuracy = 0.6875, precision = 0.5417, recall = 0.6875, F1 score = 0.5917
Query ID 419/475
Support set metrics: Loss = 0.9248, accuracy = 0.7125, precision = 0.6684, recall = 0.6800, F1 score = 0.6569
Query set metrics: Loss = 1.2653, accuracy = 0.6250, precision = 0.7500, recall = 0.6667, F1 score = 0.6667
Query ID 420/475
Support set metrics: Loss = 0.9296, accuracy = 0.7125, precision = 0.7231, recall = 0.7100, F1 score = 0.6916
Query set metrics: Loss = 1.0711, accuracy = 0.6875, precision = 0.4259, recall = 0.5556, F1 score = 0.4741
Query ID 421/475
Support set metrics: Loss = 1.1331, accuracy = 0.6750, preci

Support set metrics: Loss = 1.1433, accuracy = 0.7000, precision = 0.7268, recall = 0.7000, F1 score = 0.7006
Query set metrics: Loss = 0.8591, accuracy = 0.6875, precision = 0.8095, recall = 0.7143, F1 score = 0.7286
Query ID 453/475
Support set metrics: Loss = 1.1647, accuracy = 0.6125, precision = 0.6707, recall = 0.5700, F1 score = 0.5791
Query set metrics: Loss = 1.3452, accuracy = 0.6250, precision = 0.7917, recall = 0.7292, F1 score = 0.7250
Query ID 454/475
Support set metrics: Loss = 1.0644, accuracy = 0.6625, precision = 0.7235, recall = 0.6600, F1 score = 0.6828
Query set metrics: Loss = 0.7065, accuracy = 0.8125, precision = 0.7000, recall = 0.8000, F1 score = 0.7381
Query ID 455/475
Support set metrics: Loss = 0.9542, accuracy = 0.7375, precision = 0.7414, recall = 0.7300, F1 score = 0.7255
Query set metrics: Loss = 0.7157, accuracy = 0.8125, precision = 0.8148, recall = 0.8148, F1 score = 0.8074
Query ID 456/475
Support set metrics: Loss = 0.9142, accuracy = 0.7250, preci

In [11]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aMEL-dssplit-order4/OML-order4-id4-2023-09-29_03-37-26.455645_update5_results.csv


In [12]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aMEL-dssplit-order4/OML-order4-id4-2023-09-29_03-37-26.455645_update5_time_inference.csv
